In [2]:
import pandas as pd
import numpy as np


import gzip
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora, models, similarities
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.ldamodel import LdaModel
import pickle
import pyLDAvis.gensim as gensimvis
import pyLDAvis

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Spectral6
from ipywidgets import *
import requests
import ujson as json
import random

In [4]:
df.head()

,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall,reviewTime,summary
0,A1EWN6KZ4HMLT7,011040047X,S. Prescott,"[2, 2]",1344902400,"The case pictured is a soft violet color, but ...",1,"08 14, 2012",Wrong color
1,A1YX2RBMS1L9L,0110400550,Andrea Busch,"[0, 0]",1353542400,Saw this same case at a theme park store for 2...,5,"11 22, 2012",Great product
2,A180NNPPKWCCU0,0110400550,Aniya pennington,"[3, 3]",1374105600,case fits perfectly and I always gets complime...,5,"07 18, 2013",Perfect
3,A3HVRXV0LVJN7,0110400550,BiancaNicole,"[4, 4]",1358035200,Best phone case ever . Everywhere I go I get a...,5,"01 13, 2013",A++++
4,A292527VPX98P8,0110400550,Cebell,"[0, 1]",1353888000,It may look cute. This case started off pretty...,1,"11 26, 2012",Do NOT GET IT!!


In [3]:
asins = dict()
categories = dict()
reviewtextlist = list()
review_categories = list()

In [4]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [5]:
def readMetaData(path,catname):
    i = 0
    for d in parse(path):
        numCategories=0
        title= "Not available"
        if 'title' in d:
            title=d['title']
        if(len(d['categories']) > 0):
            numCategories = len(d['categories'][0])
            for i in  range(0,numCategories):
                cat = d['categories'][0][i]
                if cat not in categories:
                    categories[cat]=cat
                    review_categories.append(cat)
                if cat == catname:
                    asins[d['asin']] = title
        i += 1

In [6]:
def readReviewdata(path):
    with gzip.open(path, "rb") as f:
        for line in f:
            # print line
            d = eval(line)
            reviewerId = d['reviewerID']
            asin = d['asin']
            rating = d['overall']
            summary = d['summary']
            reviewText = d['reviewText']
            dtime = d['reviewTime']
            if asin in asins and len (reviewText)>0:
                reviewtextlist.append(reviewText)

In [7]:
import pandas as pd 
import gzip 

def parse(path): 
    g = gzip.open(path, 'rb') 
    for l in g: 
        yield eval(l) 
        
def getDF(path): 
    i = 0 
    df = {} 
    for d in parse(path): 
        df[i] = d 
        i += 1 
    return pd.DataFrame.from_dict(df, orient='index') 

In [22]:
df = getDF('reviews_Cell_Phones_and_Accessories.json.gz')
pickle.dump(df,open("dataframe10000.p","wb"))

In [8]:
df = pickle.load(open( "dataframe10000.p", "rb" ))
df.head()

,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall,reviewTime,summary
0,A1EWN6KZ4HMLT7,011040047X,S. Prescott,"[2, 2]",1344902400,"The case pictured is a soft violet color, but ...",1.0,"08 14, 2012",Wrong color
1,A1YX2RBMS1L9L,0110400550,Andrea Busch,"[0, 0]",1353542400,Saw this same case at a theme park store for 2...,5.0,"11 22, 2012",Great product
2,A180NNPPKWCCU0,0110400550,Aniya pennington,"[3, 3]",1374105600,case fits perfectly and I always gets complime...,5.0,"07 18, 2013",Perfect
3,A3HVRXV0LVJN7,0110400550,BiancaNicole,"[4, 4]",1358035200,Best phone case ever . Everywhere I go I get a...,5.0,"01 13, 2013",A++++
4,A292527VPX98P8,0110400550,Cebell,"[0, 1]",1353888000,It may look cute. This case started off pretty...,1.0,"11 26, 2012",Do NOT GET IT!!


In [9]:
df_meta = getDF('meta_Cell_Phones_and_Accessories.json.gz')
pickle.dump(df_meta, open("dataframe_meta_10000.p","wb"))

In [10]:
df_meta = pickle.load(open( "dataframe_meta_10000.p", "rb" ))

In [11]:
#df_meta.head()
cat_list = [df_meta['categories'][i][0][len(df_meta['categories'][i][0])-1] for i in range(len(df_meta))]
print len(cat_list)
print len(list(set(cat_list)))

346793
81


In [206]:
print sorted(list(set(cat_list)))

['AC Adapters', 'Accessories', 'Accessories & Supplies', 'Accessory Kits', 'Appliances, Heating, A/C, & Ventilation', 'Armbands', 'Audio Adapters', 'Basic Cases', 'Bathing & Skin Care', 'Batteries', 'Batteries, Chargers & Accessories', 'Battery Charger Cases', 'Battery Chargers', 'Bluetooth Headsets', 'Bluetooth Speakers', 'Camera & Photo', 'Cameras', 'Car Chargers', 'Car Cradles', 'Car Cradles & Mounts', 'Car Kits', 'Car Mounts', 'Car Speakerphones', 'Cases', 'Cases & Sleeves', 'Cell Phone Accessories', 'Cell Phone Docks', 'Cell Phones', 'Cell Phones & Accessories', 'Changing Tables', 'Chargers', 'Chargers & Adapters', 'Clothing', 'Computer Components', 'Computers & Accessories', 'Connected Devices', 'Contract Cell Phones', 'Customizable Cases', 'Data Cables', 'Data Cards', 'Door & Seat Back Organizers', 'Electronics & Gadgets', 'External Battery Packs', 'Family Life', 'Fan Shop', 'Flattening Irons', 'Gun Holsters, Cases & Bags', 'Harnesses & Leashes', 'Headsets', 'Holsters & Clips', 

In [18]:
pcat_list = [df_meta['categories'][i][0][len(df_meta['categories'][i][0])-2] for i in range(len(df_meta))]
print len(list(set(pcat_list)))

41


In [208]:
print sorted(list(set(pcat_list)))

['Accessories', 'Accessories & Supplies', 'Baby Products', 'Backpacks & Carriers', 'Bags, Packs & Accessories', 'Bath & Body', 'Batteries', 'Batteries & Accessories', 'Batteries, Chargers & Accessories', 'Books', 'CDs & Vinyl', 'Car Accessories', 'Car Cradles & Mounts', 'Cases', 'Cell Phones', 'Cell Phones & Accessories', 'Chargers', 'Computers & Accessories', 'Connected Devices', 'Consoles & Organizers', 'Cradles, Mounts & Stands', 'Craft Supplies', 'Cutting Tools', 'Electronics', 'Fan Shop', 'Furniture', 'Growing Up & Facts of Life', 'Headsets', 'Hunting', 'Irons', 'Jump Starters, Battery Chargers & Portable Power', 'MP3 Player Accessories', 'Mobile Broadband', 'No-Contract Cell Phones', 'PDA & Handheld Accessories', 'RV Parts & Accessories', 'Safety', 'Sports & Outdoors', 'Strollers', 'Touch Screen Tablet Accessories', 'Underwater Photography']


In [19]:
df_meta['subcat1'] = pcat_list
df_meta['subcat2'] = cat_list
#del df_meta['subact2']
df_meta.head()

,asin,categories,title,price,salesRank,imUrl,brand,related,description,subcat1,subcat2
0,0110400550,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Pink &amp; White 3d Melt Ice-cream Skin Hard C...,3.33,{u'Cell Phones & Accessories': 83460},http://ecx.images-amazon.com/images/I/31zn6SOL...,,"{u'also_bought': [u'B00C56IXFG', u'B008ZUQWOK'...",Pink & White 3D Melt Ice-Cream Skin Hard Case ...,Cases,Basic Cases
1,011040047X,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Purple Hard Case Cover for Iphone 4 4s 4g with...,1.94,{u'Cell Phones & Accessories': 495795},http://ecx.images-amazon.com/images/I/41WCZc2d...,NaN,"{u'buy_after_viewing': [u'B008RU7UL2', u'B0069...",Purple Hard Case Cover for iPhone 4 4S 4G With...,Cases,Basic Cases
2,0195866479,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Hello Kitty Light-weighted Chrome Case Black C...,2.94,{u'Cell Phones & Accessories': 371302},http://ecx.images-amazon.com/images/I/41fy1%2B...,NaN,"{u'buy_after_viewing': [u'B00530RXP2', u'B004S...","Thin and light weighted,\nCase's unique design...",Cases,Basic Cases
3,0214514706,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Cool Summer Breeze in the Ocean Beach Collecti...,0.94,{u'Cell Phones & Accessories': 778100},http://ecx.images-amazon.com/images/I/415cmp6Q...,NaN,"{u'buy_after_viewing': [u'B0042FV2SI', u'B0086...",Product Name: Cool Summer Breeze In The Ocean...,Cases,Basic Cases
4,0214714705,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Cool Summer Breeze In The Ocean Beach Collecti...,5.79,{u'Cell Phones & Accessories': 654894},http://ecx.images-amazon.com/images/I/41XDwPt2...,NaN,"{u'buy_after_viewing': [u'B008EU7HRM', u'B0086...",Product Name: Cool Summer Breeze In The Ocean...,Cases,Basic Cases


In [29]:
import difflib
from difflib import SequenceMatcher
import math

string = 'Plantronics CS530 Office Wireless Headset with Extended Microphone'
name = string.lower()
price = 182.17
titles = [title for title in df_meta['title'][df_meta['price']<= price + 10.0][df_meta['price']>=max(0,price-10.0)]]
tuples = []
for title in titles:
        m = SequenceMatcher(None, str(title).lower(), name)
        tuples.append((m.ratio(), title))
        
sorted_tuples = sorted(tuples, key=lambda x: x[0], reverse=True)
print sorted_tuples

[(0.6782608695652174, 'Plantronics CS530 Wireless Headset System - Black'), (0.625, 'Plantronics-CS540 Convertible Wireless Headset'), (0.5874125874125874, 'Plantronics CS510 - Over-the-Head monaural Wireless Headset System - DECT 6.0'), (0.5299145299145299, 'Plantronics Digital Tty with Text Answering Machine'), (0.5161290322580645, 'Jabra GN9125 Mono Soundtube Wireless Headset for Deskphone'), (0.5054945054945055, 'Plantronics Spare Headset'), (0.5, 'AKG K830BT High-End Wireless Headset with Bluetooth (Black) (Discontinued by Manufacturer)'), (0.47674418604651164, 'Creative Aurvana Gold Wired/Wireless Headset with 40mm Drivers, Bluetooth 3.0, NFC and Built-In Microphone'), (0.4666666666666667, 'Plantronics Voyager Legend CS Bluetooth Headset for Mobile Phones - Retail Packaging'), (0.45595854922279794, 'Plantronics SAVI W440 Convertible, Wireless DECT Headset System for Laptop, Softphone &amp; Multimedia Use - Black &amp; Silver'), (0.41379310344827586, 'SONY ERICSSON XPERIA RAY PREM

In [23]:
prod_asin = df_meta[df_meta['title']=="Plantronics CS530 Wireless Headset System - Black"]['asin']
asin_list =  prod_asin.tolist()
#print asin_list[0]
review_list = df['reviewText'][df['asin']==asin_list[0]]
rating_list = df['overall'][df['asin']==asin_list[0]]
asin_table = pd.DataFrame()
asin_table['review'] = review_list
asin_table['rating'] = rating_list
asin_table.sort_values(["rating"], inplace=True, ascending=False)  
asin_table.reset_index(drop=True, inplace=True)
#print asin_table.head()
#print len(asin_table)
asin_reviews = asin_table['review'].tolist()
pos_reviews = asin_reviews[0:80]
neg_reviews = asin_reviews[81:]

pos_reviews

Plantronics CS530 Wireless Headset System - Black
B0082DCPQ4


['I got 2 of these for myself and my receptionist and we both LOVE them. They are very convenient, the quality is great, and they are comfortable to boot!',
 'Easy set up and works great all around the office. I have a 6000 square foot office and have not lost a call. Great call quality.',
 'I have used 2 of these throughout my career....they are amazing and take all the strain off your neck when on the phone',
 "Everything I wanted. The range and sound are both great. I already was used to the fit of the older version, and this one fits just the same, so I found it comfortable. The base looks slick too, and I love the magnetic dock. No more coming into the office in the morning to an uncharged headset because I didn't seat it in the charger properly the night before.",
 'Very nice product. Excellent reception. Works well in office atmosphere with no interference with other. I would recommend to all.',
 'I use this item in an office setting.  I have tried a few other brands and found t

In [15]:
reviews = pickle.load(open("reviews_10000.p","rb"))

In [ ]:
model = LdaModel.load('headsets_10000.lda')
lda_topics = pickle.load(open("headsets_topics_from_10000.p", "rb"))
corpus = pickle.load(open("corpus_10000.p","rb"))
dictionary = pickle.load(open("dic_10000.p","rb"))

In [56]:
# extract all review sentences that contains the term - ear
from __future__ import unicode_literals
import spacy
from spacy.en import English
import re

doc = asin_reviews
ear_sents = []
for item in doc:
    document = unicode(str(item), encoding='utf-8')
    parser = English()
    parsed_doc = parser(document)
    earlist = [sent for sent in parsed_doc.sents
                       if 'ear' in sent.string.lower()]
    ear_sents += earlist
print ear_sents

[I've been buying plantronics for years, and love their product.  , Slick and light, really good range and you can hear well from it., As well, it will randomly go into dead-air but the person is still on the other line and can hear you- it will just do that randomly., I can do whatever I'm doing with my hands and when I received a phone call I just touch a button at my ear and start talking., . I would know because I have a very close, hands on use on use for this as it stays on my ear about 45 hours a week., It is comfortable to wear and you can clearly hear every conversation that comes through., loved the older model of this but after a couple of years the band at the very top started to break and I was losing phone connections so I got this model  the new version operates just as the older model but with a major improvement in the base.  , My previous model lasted 10 years until electric surge did it in., So far I love the headset although the mute feature takes nearly 4 or 5 seco

In [ ]:
earlist = ear_sents
pickle.dump(earlist, open("ear_sentences_10000.p", "wb"))

In [29]:
doc = asin_reviews
pos_tuples = []
neg_tuples = []
pos_doc = []
neg_doc =[]

for i in range(len(asin_table)):
    if asin_table['rating'][i] >= 4.0:
        pos_doc.append(asin_table['review'][i])
        pos_tuples.append((asin_table['rating'][i],asin_table['review'][i]))
    else:
        neg_doc.append(asin_table['review'][i])
        neg_tuples.append((asin_table['rating'][i],asin_table['review'][i]))

print neg_tuples

[(3.0, 'this product is ok.  not great or poor.  the design is great.  sound quality is good but kind of a hassle to line up the charging.  Had some trouble so I returned it'), (3.0, "I love having the room to move around and talk while I work.  But the earpiece occasionally shrieks in my ear.  It's not completely horrible, I can deal with it.  But I was expecting more.  Especially for the price."), (3.0, 'The comfort is there, but the performance is not. I had an older Plantronics headset that worked far better and the range was farther as well. This one you can only walk MAYBE 7 ft from the base before distortion and loosing signal or even dropping the call. I would look at another headset before considering this one. Go back to the drawing board Plantronics and try again!'), (1.0, "Hard to review a product that doesn't work. I'm currently returning it for a replacement and will see if that one works."), (2.0, "I was VERY disapointed to find out this unit needs yet ANOTHER part ($89.

In [ ]:
# extract all review sentences that contains the term - sound
from __future__ import unicode_literals
import spacy
from spacy.en import English
import re 

pos_sound_sents = []
for item in pos_tuples:
    document = unicode(str(item[1]), encoding='utf-8')
    parser = English()
    parsed_doc = parser(document)
    soundlist = [(item[0],sent) for sent in parsed_doc.sents
                       if 'sound' in sent.string.lower()]
    pos_sound_sents += soundlist
print pos_sound_sents

In [ ]:
# extract all review sentences that contains the term - sound
from __future__ import unicode_literals
import spacy
from spacy.en import English
import re

#doc = reviews[0:9]
doc = neg_reviews[0:10]
neg_sound_sents = []
for item in doc:
    document = unicode(str(item), encoding='utf-8')
    parser = English()
    parsed_doc = parser(document)
    soundlist = [sent for sent in parsed_doc.sents
                       if 'sound' in sent.string.lower()]
    neg_sound_sents += soundlist
print neg_sound_sents